## imports, etc.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import torch
import torchvision
from tqdm.notebook import tqdm
import os
from sklearn.semi_supervised import LabelPropagation
from sklearn.neural_network import MLPClassifier

In [2]:
np.random.seed(0)

In [3]:
X_train = torch.load("data/FashionMNIST_train_latent.pt")
X_test = torch.load("data/FashionMNIST_test_latent.pt")
y_train = torch.load("data/FashionMNIST_train_labels.pt")
y_test = torch.load("data/FashionMNIST_test_labels.pt")

In [4]:
os.chdir("../fashion_ml")

In [5]:
import warnings
warnings.filterwarnings("ignore")

## limit data labels

In [6]:
X = torch.cat((X_train, X_test), 0)
y = torch.cat((y_train, y_test), 0)

discard all but 200 random labels (we can increase the number but I feel like 1000+ doesn't really count as semi-supervised anymore)

In [7]:
n_labeled = 200

In [8]:
ind = np.random.randint(len(X), size=(n_labeled))
y_lim = np.repeat(-1, len(y))
y_lim[ind] = y[ind]

## semi-supervised clustering

search for optimal k: 2 is optimal, anything over 3 predicts everything as the first class. overall clustering does not work well at all.

In [9]:
def test_knn(n):
    lpm = LabelPropagation(kernel='knn',n_neighbors=n,n_jobs=-1,max_iter=5000)
    lpm.fit(X, y_lim)
    print(lpm.score(X, y))
    y_pred = lpm.predict(X)
    print(np.unique(y_pred))
    return lpm

In [10]:
test_knn(2)

0.10241428571428571
[0 1 2 3 4 5 6 7 8 9]


LabelPropagation(kernel='knn', max_iter=5000, n_jobs=-1, n_neighbors=2)

In [11]:
test_knn(3)

0.10145714285714286
[0 1 2 3 4 5 6 7 8 9]


LabelPropagation(kernel='knn', max_iter=5000, n_jobs=-1, n_neighbors=3)

In [12]:
test_knn(7)

0.1
[0]


LabelPropagation(kernel='knn', max_iter=5000, n_jobs=-1)

In [13]:
test_knn(12)

0.1
[0]


LabelPropagation(kernel='knn', max_iter=5000, n_jobs=-1, n_neighbors=12)

final clusters:

In [14]:
lpm = LabelPropagation(kernel='knn',n_neighbors=2,n_jobs=-1,max_iter=5000)
lpm.fit(X, y_lim)
y_pred = lpm.predict(X)

## mlp models

pull out train vs. test:

In [15]:
X_train = X[y_lim != -1]
X_test = X[y_lim == -1]
y_train = y[y_lim != -1]
y_test = y[y_lim == -1]

test a few networks:

In [35]:
clf = MLPClassifier(hidden_layer_sizes=(20,5), tol=1e-5)
clf.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(20, 5), tol=1e-05)

In [36]:
clf.score(X_test, y_test)

0.3267908309455587

In [43]:
clf = MLPClassifier(hidden_layer_sizes=(100,20,5), tol=1e-4)
clf.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(100, 20, 5))

In [44]:
clf.score(X_test, y_test)

0.4045845272206304

In [59]:
clf = MLPClassifier(hidden_layer_sizes=(200,100,25,5), tol=1e-5)
clf.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(200, 100, 25, 5), tol=1e-05)

In [60]:
clf.score(X_test, y_test)

0.5203581661891118